# Problem Description

This project implements Reinforcement Learning (RL) for the Recycling Robot problem (Example 3.3 in the textbook).

Goal:

- Use the Temporal Difference (TD) algorithm so the robot learns to maximize the total reward over time.

Key choices and requirements:

- Select values for the probabilities alpha (α) and beta (β), and for the rewards r_search and r_wait, ensuring r_search > r_wait.
- Use TD learning to update value estimates and derive a policy that maximizes accumulated reward.
- Define a learning period (epoch) of, for example, 1000 steps. After each epoch, save the total reward accumulated during that epoch to a file named `rewards.txt`.
- Train for multiple epochs and plot the accumulated total reward over epochs (Matplotlib or Seaborn). Optionally repeat training runs and average the curves to smooth results.
- Visualize the learned (optimal) policy as a heatmap (Seaborn), where each cell reflects the preferred action(s) for each robot state.

See `README.md` for the short project title and overview.

In [ ]:
python /home/marcelo_baez/phd/project-1-RL-EMAP/recycling_robot.py
